In [1]:
import os, sys

from datetime import datetime, timedelta
import time

from inlayr.provider import get_provider

### Init

In [2]:
provider = get_provider(
    chain = "evm",
    chain_params = {
        "wallet_pk": open(os.path.join(os.environ.get("inlayr_root"), r"keys\ethereum_wallet")).read(),
    },
    
    rpc = "onerpc", 
    rpc_params = {
        "chain_id": 1,
    }, 
    
    aggregator = "oneinch",
    aggregator_params = {
        "headers": {
            "Authorization": "Bearer " + open(os.path.join(os.environ.get("inlayr_root"), r"keys\oneinch-api-key")).read(),
            "accept": "application/json",
        }
    }
)

### Inputs

In [3]:
source_token = "0xdac17f958d2ee523a2206206994597c13d831ec7" # USDT mint address
destination_token = "0xec53bf9167f50cdeb3ae105f56099aaab9061f83" 

source_amount = int(129 * 1e6)
slippage = int(0.05 * 10000) # Convert to bps

### Set allowance

In [237]:
approval_transaction = provider.get_approval(
    source_token = source_token,
    source_amount = source_amount
)

Allowance is sufficient, no approval needed.


In [238]:
if approval_transaction.transaction is not None:
    approval_executed = provider.execute_trx(
        transaction = approval_transaction
    )

In [239]:
if approval_transaction.transaction is not None:
    display(
        provider.trx_status(
            signature = approval_executed
        )
    )

### Execute swap

In [240]:
quote = provider.get_quote(
    source_token = source_token, 
    source_amount = source_amount, 
    destination_token = destination_token, 
    slippage = slippage
)

print ("max gas fee:", (1.0e-18) * int(quote.quote['tx']['gas']) * int(quote.quote['tx']['gasPrice']), "ETH")

max gas fee: 9.172536292781999e-05 ETH


In [241]:
swap_transaction = provider.get_swap(
    quote = quote
)

In [242]:
swap_executed = provider.execute_trx(
    transaction = swap_transaction
)

In [ ]:
provider.trx_status(
    signature = swap_executed
)

### Check balance

In [4]:
token_position = provider.get_balance(
    source_token = destination_token
)

token_position

Balance(source='EVM', amount=281837695505214370218)